# Instructions

This colab contains code used to generate the wordcloud visualization for the Real-World Task experiments (generating social media updates) on the various LLMs experimented on in the paper "Personality Traits in Large Language Models" (https://arxiv.org/pdf/2307.00184). The code uses the "scored" LLM outputs (pickled dataframes) from the Independent Trait Shaping Analysis notebook, as well as the actual social media updates generated by the LLMs (in their own pickled df), along with the AMS scores (CSV file) resulting from running the Apply Magic Sauce model on the generated updates. The code below assumes that all the data produced and consumed in the colab (especially the pickled dataframe outputs of running inference on various LLMs) lives in a local filesystem either in a cloud instance running a Jupyter notebook such as Google Colab or a desktop. But those file I/O operations can easily be replaced to use any other file management solutions. The inline comments for some of the operations explain the motivation behind them and what to expect in the results of running an analysis in a cell.

To run this colab:
1. Connect to an appropriate runtime. (For instance, if running the bulk inference directly from the colab, connect to a GPU kernel.)
2. Check experiment parameters below.
3. Run the code cells for analysis.

# Setup

## Imports

In [ ]:
#@title Install dependencies for Downstream task analysis and visualization
%pip install pandas
%pip install matplotlib
%pip install wordcloud

In [ ]:
#@title Load dependencies
import pandas as pd
import matplotlib.pyplot as plt
import wordcloud

In [ ]:
#@title File locations setup  { run: "auto" }

#@markdown `ABL01_SCORES` is the filename of pickled results of the output from
#@markdown the Independent Trait Shaping Analysis notebook.
#@markdown This "scored session dataframe "is an input for this colab.
ABL01_SCORES = 'sample_pkl_file.pkl'  # @param {"type":"string"}

#@markdown `STATUS_UPDATES_DATA` is the file path of the output from the LLMs
#@markdown generating the downstream task (social media updates) data based on
#@markdown their shaped traits.
STATUS_UPDATES_DATA = 'sample_downstream_data.pkl'  # @param {"type":"string"}

#@markdown `AMS_DATA` is the path to the results of running AMS on the file above.
AMS_DATA = 'sample_ams_output.csv'  # @param {"type":"string"}

#@markdown Path of the file where the dataframe created by joining the above three
#@markdown pieces of data can be stored as a pickled dataframe.
SAVE_SCORES_FILENAME = 'sample_scored_dataframe.pkl'  # @param {"type":"string"}

#@markdown Whether the model who's data is being analyzed is a PaLM model variant or not?
#@markdown Some of the pre-processing on the input dataframe differs between PaLM and non-PaLM models.
IS_PALM_MODEL = True  # @param {"type":"boolean"}

## Constants

In [ ]:
IPIP_SCALE_IDS = [
    'IPIP300-EXT',
    'IPIP300-AGR',
    'IPIP300-CON',
    'IPIP300-NEU',
    'IPIP300-OPE'
]

AMS_SCALE_IDS = [
    'ams-IPIP300-EXT',
    'ams-IPIP300-AGR',
    'ams-IPIP300-CON',
    'ams-IPIP300-NEU',
    'ams-IPIP300-OPE'
]

AMS_SCALE_IDS_2 = [
    'BIG5_Extraversion',
    'BIG5_Agreeableness',
    'BIG5_Conscientiousness',
    'BIG5_Neuroticism',
    'BIG5_Openness'
]

## Read in Data

In [ ]:
# independent shaping personality test scores
# 2250 profiles x 300 items = 675k rows
test_scores = pd.read_pickle(ABL01_SCORES)

# downstream task social media status updates
# 2250 profiles x 25 repeats = 56,250 rows
status_updates_raw = pd.read_pickle(STATUS_UPDATES_DATA)

# AMS personality predictions based on status updates
# 7 values per profile
ams_predictions_raw = pd.read_csv(AMS_DATA, index_col=0)

In [ ]:
if not IS_PALM_MODEL:
  # pre-process status updates data
  # consolidate every 25 updates under their prompted personality profile
  status_updates = status_updates_raw.groupby('item_preamble_id')['model_output'].agg(list).reset_index()
  status_updates['model_output'] = status_updates['model_output'].apply(lambda x: '\n'.join(x))
else:
  status_updates = status_updates_raw

In [ ]:
status_updates.info()

In [ ]:
# pre-process AMS data

# create ID for every 7 rows
ams_predictions_raw['ID'] = ams_predictions_raw.index // 7

# pivot to wide
ams_predictions_wide = ams_predictions_raw.pivot(
    index=['ID', 'user_id'],
    columns='trait',
    values='value')

# average AMS scores by shared prompt
# 56,250 rows / 25 repetitions -> 2,250 rows

# group by `item_preamble_id` (labeled `user_id` here)
ams_predictions = ams_predictions_wide.groupby('user_id').agg('mean')

In [ ]:
ams_predictions['user_id'] = ams_predictions.index
ams_predictions

## Join Data

In [ ]:
# attach ablation 01 scores to status updates
# create partial IDs for matching
status_updates['partial_id'] = status_updates['item_preamble_id'].str[:-4]
test_scores['partial_id'] = test_scores['item_preamble_id'].str[:-4]
ams_predictions['partial_id'] = ams_predictions['user_id'].str[:-4]

# drop columns
status_updates.drop('item_preamble_id', axis=1, inplace=True)
test_scores.drop('item_preamble_id', axis=1, inplace=True)
ams_predictions.drop('user_id', axis=1, inplace=True)

if IS_PALM_MODEL:
  df_grouped = pd.merge(pd.merge(test_scores, ams_predictions, on='partial_id'), status_updates, on='partial_id')
else:
  dfs = [status_updates, test_scores, ams_predictions]
  dfs = [df.set_index('partial_id') for df in dfs]
  df_grouped = dfs[0].join(dfs[1:])


In [ ]:
# add intended personality level info
LVL_IDS = ['lvl-EXT', 'lvl-AGR', 'lvl-CON', 'lvl-NEU', 'lvl-OPE']
df_grouped['level_info'] = df_grouped['partial_id'] if IS_PALM_MODEL else df_grouped.index
df_grouped[LVL_IDS + ['description_id']] = df_grouped['level_info'].str.split('-', expand=True)

In [ ]:
def extract_integer(s):
  """Extract the level integer contained in a string."""
  result = ''
  for char in s:
    if char.isdigit():
      result += char
  try:
    return int(result)
  except ValueError:
    return None

In [ ]:
df_grouped[LVL_IDS + ['description_id']] = df_grouped[LVL_IDS + ['description_id']].map(extract_integer)

In [ ]:
# peek at new merged dataframe
# should be 2,250 rows
df_grouped

In [ ]:
group = df_grouped

In [ ]:
# optional: save scores to disk
if SAVE_SCORES_FILENAME:
  df_grouped.to_pickle(SAVE_SCORES_FILENAME)

##Plot Wordclouds

In [ ]:
# Create a 2x5 layout for subplots
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(20, 10))

# Flatten the axes array
axes = axes.flatten()

stopwords = set()
for w in wordcloud.STOPWORDS:
  stopwords.add(w)

for i, dimension in enumerate(['ext', 'agr', 'ope', 'con', 'neu']):
  for j, level in enumerate([1, 9]):
    # Concatenate all the values in the 'Text' column
    text = ' '.join(group[group['partial_id'].str.contains(f'{dimension}{level}')]['model_output'].tolist())
    # Create a WordCloud object
    wc = wordcloud.WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(text)
    # Set the title for the subplot
    axes[i*2 + j].set_title(f'{dimension}{level}')
    # Plot the word cloud in the corresponding subplot
    axes[i*2 + j].imshow(wc, interpolation='bilinear')
    axes[i*2 + j].axis('off')
# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()
# Plot the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()